## Hop Plot WCC
Here's the idea
1. Same exact thing as Hop Plot except should be done on the largest WCC
- Two nodes are weakly connected if there exists an undirected path from node u to v
- So need to find the largest WCC
- Going to use NetworkX: https://networkx.org/documentation/stable/tutorial.html in order to find the largest WCC so I don't have to reinvent the wheel
- Plan: Turn CSV into a list of directed edges then add all edges to Graph G, then run hop plot on this graph
2. See Hop Plot code for idea on Hop Plot

In [1]:
import networkx as nx
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("web-stanford.txt", sep = "\t")

FileNotFoundError: [Errno 2] File b'RandomNodeEdge.csv' does not exist: b'RandomNodeEdge.csv'

In [23]:
#add edges to an edge list
edge_list = []
for i in range(1, len(data) + 1):
    start = int(data[i-1:i]["FromNode"])
    end = int(data[i-1:i]["ToNode"])
    edge_list.append((start, end))

In [49]:
#create our directed graph and set of weakly connected components
G_smp = nx.DiGraph()
G_smp = nx.from_pandas_edgelist(input_data, "FromNode", "ToNode", create_using=nx.DiGraph())
largestcc = max(nx.weakly_connected_components(G_Original), key=len)

In [54]:
list_of_nodes = np.unique(np.array(data["FromNode"]))
#just edit our graph now to say if FromNode or ToNode is not in our largestCC get rid of the edge

In [59]:
#Puts a dictionary together of nodes and their outlinks 1 step away except filters now based on if it's in the WCC (a well smaller graph)
edge_holder = {}
for nodes in list_of_nodes:
    if(nodes not in largestcc):
        continue
    outlinks = list(data[data["FromNode"] == nodes]["ToNode"])
    filtered = list(filter(lambda x: x in largestcc, outlinks))
    if(len(filtered) > 0):
        edge_holder[nodes] = set(filtered)

In [62]:
def hopping(h): #now this returns all edges that can be reached in some amount of hops
    #this works for two steps now need to make it work for 3,4, etc...
    #first step is for each h create a new "edge_holder" that holds the elements it's connected ot in h steps
    edge_holder2 = edge_holder.copy()
    for nodes in edge_holder:
        last_nodes = edge_holder[nodes]
        hops = h
        while(hops > 1):
            temp = set()
            for outlinks in last_nodes:
                if(outlinks in edge_holder):
                    temp = set.union(temp, edge_holder[outlinks])            
            edge_holder2[nodes] = set.union(temp, edge_holder2[nodes])
            hops = hops - 1
            last_nodes = temp
    
    
            
    return edge_holder2

In [63]:
def counting(num_steps):
    e = hopping(num_steps);
    pairs = 0;
    for nodes in e:
        for outlinks in e[nodes]:
            if(outlinks in e and nodes in e[outlinks]):
                pairs += 1;
    
    return int(pairs/2); #since we end up double counting

In [66]:
counting(4) #hopplot done - there are some interesting things to note here especially in WCC

0

## End Here